In [1]:
from contrans import contrans
import numpy as np
import pandas as pd

In [2]:
ct = contrans()
ct.mypassword

'dataengineering1'

In [3]:
votes = ct.get_votes()
votes

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
486659,118,House,1118,31101,6,100.0
486660,118,House,1118,31102,9,99.6
486661,118,House,1118,39301,6,100.0
486662,118,House,1118,39307,6,100.0


In [6]:
house = house[['rollnumber', 'icpsr', 'cast_code']]

NameError: name 'house' is not defined

In [ ]:
house_mat = pd.merge(house, house, on='rollnumber')
house_mat

NameError: name 'house' is not defined

In [5]:

house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y']).agg({'agree':'mean'})
house_mat = house_mat.reset_index()
house_mat

NameError: name 'house_mat' is not defined

In [7]:
ideo = ct.get_ideo()
members = ideo[['icpsr', 'party_code', 'bioname']]
members

AttributeError: 'contrans' object has no attribute 'get_ideo'